In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [ ]:
# load library doing the computing
import wte
import bz2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [ ]:
# basic variables -- adjust these for the environment
base_dir = "data/" # base directory where all data files are
# directory for travel time and index matrices
fnbase = base_dir + "nyc_travel_times"
# file with travel distances
dist_fn = base_dir + "nyc_travel_times/distances_dir.bin"
# size of the matrices (i.e. nodes in the network)
matrix_size = 4092
# trips to serve in the simulation -- one example day
tripsfile = base_dir + "nytrips/nytrips_15180.bz2"
# trips to serve in the simulation -- base filename for all days
tripsfile_base = base_dir + "nytrips/nytrips"
# number of drivers to use (only in the below examples)
ndrivers = 6000
# maximum waiting time (in seconds)
tmax = 300
# file containing the frequency distribution of trip start nodes 
# this can be used as a simple way to relocate drivers in accordance with the overall demand
cruising_target_dist_fn = base_dir + "nytrips/trip_start_dist.dat"

## Basic cases
Simulation of one day, with a fixed number of drivers, measure the number of trips successfully served within 5 min

### 1. Simple run, load the trips with the c++ code, no cruising (drivers stay in place after every trip)

In [ ]:
w = wte.wait_time_estimator()
w.load_index_default_fn(4092, fnbase, use_paths = True)
w.load_distances(dist_fn)
# note: we filter out too short and too long trips
w.read_trips_csv(tripsfile, "bz2", length_min = 120, length_max = 7200,
                 dist_min = 500, speed_max = 100 / 3.6)

# add the drivers
w.add_drivers(ndrivers, 0)

N = w.ntrips
nserved = 0
for i in tqdm_notebook(range(N)):
    trip = w.get_trip(i)
    if w.process_trip(trip, tmax):
        nserved += 1

total_trip = 0
total_empty = 0
for d in w.drivers:
    total_trip += d['total_trip_distance']
    total_empty += d['total_empty_distance']
        
print('Served {} / {} trips\n'.format(nserved, N))

### 2. Same, but read the trips with Pandas 
This can be useful if e.g. additional processing of trips or if there is an issue using the read_trips_csv() function

In [ ]:
trips = pd.read_csv(bz2.open(tripsfile), header = None,
                    names = ['id','start_ts','end_ts','start_node','end_node'])
# filter trips (note that this only takes into account trip length, but not trip speed as above)
trips = trips[(trips.start_node != trips.end_node) & (trips.end_ts - trips.start_ts > 120)
              & (trips.end_ts - trips.start_ts < 7200)]
trips.sort_values('start_ts', inplace = True)

N = trips.shape[0]

w = wte.wait_time_estimator()
w.load_index_default_fn(4092, fnbase, use_paths = True)
w.load_distances(dist_fn)

# add the drivers -- we cannot use add_drivers(), since that uses the distribution of trip
# start locations from trips loaded by the C++ code
avgtime = np.mean(trips.end_ts - trips.start_ts)
for i in range(ndrivers):
    node = trips.iloc[np.random.choice(N)].start_node
    t1 = np.round(np.random.exponential(avgtime))
    w.add_driver(int(t1), node)

nserved = 0
for i in tqdm_notebook(range(N)):
    if w.process_trip(trips.iloc[i], tmax):
        nserved += 1

print('Served {} / {} trips'.format(nserved, N))

### 3. Use "cruising": simple version
A static target distribution is read from a file (that contains frequencies for each node), and a relocation target is selected for each driver after finishing a trip. Once reaching the target, drivers stay in place.

In [ ]:
w = wte.wait_time_estimator()
w.load_index_default_fn(4092, fnbase, use_paths = True)
w.load_distances(dist_fn)
# note: we filter out too short and too long trips
w.read_trips_csv(tripsfile, "bz2", length_min = 120, length_max = 7200, dist_min = 500, speed_max = 100 / 3.6)

# add the drivers
w.add_drivers(ndrivers, 0)

w.strategy = wte.wait_time_estimator.cruising_strategy.DRIVER_RANDOM_NODE
w.read_cruising_target_dist(0, cruising_target_dist_fn)

N = w.ntrips
nserved = 0
for i in tqdm_notebook(range(N)):
    trip = w.get_trip(i)
    if w.process_trip(trip, tmax, True):
        nserved += 1


total_trip = 0
total_empty = 0
for d in w.drivers:
    total_trip += d['total_trip_distance']
    total_empty += d['total_empty_distance']
        
print('Served {} / {} trips\nTotal trip / empty distance: {} km / {} km'.format(
    nserved, N, total_trip / 1000, total_empty / 1000))

## Estimation of minimum fleet
Iterate the simulation until 95% of the trips are served; determine the minimum number of drivers required for this by a binary search

### 1. Helper function to load a set of trips
We typically want to use a set of trips between 5am to 5am the next day instead of between midnight. Here, we load two days and do a simple filtering.

In [ ]:
def load_trips(base_name, day, hour):
    # 1st day
    trips1 = pd.read_csv(bz2.open('{}_{}.bz2'.format(base_name, day)), header=None,
                         names=['id','start_ts','end_ts','start_node','end_node'])
    # 2nd day
    trips2 = pd.read_csv(bz2.open('{}_{}.bz2'.format(base_name, day + 1)), header=None,
                         names=['id','start_ts','end_ts','start_node','end_node'])
    # filter trips based on length
    trips1 = trips1[(trips1.start_node != trips1.end_node) & (trips1.end_ts - trips1.start_ts > 120) & (trips1.end_ts - trips1.start_ts < 7200)]
    trips2 = trips2[(trips2.start_node != trips2.end_node) & (trips2.end_ts - trips2.start_ts > 120) & (trips2.end_ts - trips2.start_ts < 7200)]
    # filter based on time of day
    trips1 = trips1[(trips1.start_ts % 86400) >= hour*3600]
    trips2 = trips2[(trips2.start_ts % 86400) < hour*3600]
    # concatenate
    trips = pd.concat([trips1, trips2])
    # ensure that the result is properly sorted
    trips.sort_values('start_ts', inplace=True)
    return trips

### 2. Helper function to process one day
Given a list of trips, this first subsamples them according to the given factor, then finds the minimum number of drivers required to serve p ratio (e.g. 95%) of trips within the given maximum waiting time (e.g. 5 min)

In [ ]:
def process_one_day(w1, rng, trips = None, pr_sub = 1.0, p_req = 0.95, twait = 300,
                    initial_step = 1000, write_progress = False):
    """
    Estimate fleet size for one instance of one day. Parameters:
        w1: a wte.wait_time_estimator class, initialized with the travel times and
            optionally with a set of trips
        rng: random number generator to use 
            (for selecting a subset of trips and assigning starting position to drivers)
        trips: a pandas.DataFrame with a set of trips to serve or None to use the trips
            loaded in w1
        pr_sub: ratio of trips to subsample before running
        p_req: required ratio of trips to serve
        twait: maximum acceptable waiting time (in seconds)
        initial_step: initial number of drivers to use
        write_progress: whether to write the current progress on each iteration
    Returns: tuple with the final fleet size, and numbers of served and unserved trips
    """
    have_trips = (trips is not None)
    N = trips.shape[0] if have_trips else w1.ntrips # total number of trips
    N1 = N
    ids = list(range(N))
    if pr_sub < 1.0:
        rng.shuffle(ids)
        N1 = int(pr_sub * N) # number of trips to use
    
    binary_search = False
    
    low = 0
    high = 2*initial_step
    fleet_size = initial_step
    
    if have_trips:
        avgtime = np.mean(trips.end_ts - trips.start_ts)
        start_time = trips.iloc[0].start_ts
        start_time -= (start_time % 3600)
        
    pct = 0.0
    matched, unmatched = 0, 0
    
    # main loop
    it = 0
    while(True):
        matched, unmatched = 0, 0
        ndrivers = fleet_size
        
        # Add drivers
        w1.reset(True)
        if have_trips:
            for i in range(ndrivers):
                x = rng.integers(N)
                node = trips.iloc[x].start_node
                t1 = np.round(rng.exponential(avgtime))
                w1.add_driver(start_time + int(t1), node)
        else:
            w1.add_drivers(ndrivers, 0)
        
        # Try to serve all selected trips
        for i in range(N):
            if ids[i] > N1:
                continue
            t1 = trips.iloc[i] if have_trips else w1.get_trip(i)
            if w1.process_trip(t1, tmax):
                matched += 1
            else:
                unmatched += 1
        
        pct = round(matched / (matched + unmatched), 3)
        if pct == p_req:
            break
        if pct < p_req:
            if binary_search:
                low = fleet_size
            else:
                # initial phase, just increase the fleet size
                low += initial_step
                high += initial_step
        else:
            high = fleet_size
            binary_search = True
        new_fleet_size = int((low + high) / 2)
        it += 1
        if write_progress:
            print('Iteration: {}, fleet size: {}, pct: {}\n'.format(it, fleet_size, pct))
        if new_fleet_size == fleet_size:
            break # no change anymore
        fleet_size = new_fleet_size
    return (fleet_size, matched, unmatched)
    

### 3. Example: run the estimation for one day and one subsampling rate

In [ ]:
w = wte.wait_time_estimator()
w.load_index_default_fn(4092, fnbase, use_paths = True)
w.load_distances(dist_fn)
w.strategy = wte.wait_time_estimator.cruising_strategy.DRIVER_RANDOM_NODE
w.read_cruising_target_dist(0, cruising_target_dist_fn)

trips = load_trips(tripsfile_base, 15000, 5)

In [ ]:
rng = np.random.default_rng()
process_one_day(w, rng, trips)

In [ ]:
# alternate version, load the trips in w -- should be significantly faster
w.read_trips_csv("{}_{}.bz2".format(tripsfile_base, 15000), compress = "bz2",
                 tmin = 18000, tlimit_in_day = True, length_min = 120, length_max = 7200,
                 dist_min = 500, speed_max = 100 / 3.6, clear_trips = True)
w.read_trips_csv("{}_{}.bz2".format(tripsfile_base, 15001), compress = "bz2",
                 tmax = 17999, tlimit_in_day = True, length_min = 120, length_max = 7200,
                 dist_min = 500, speed_max = 100 / 3.6, clear_trips = False)
w.sort_trips()

process_one_day(w, rng, None)

### 4. Run the estimation for three weeks of data and a set of subsampling rates
Note: this is only one realization. For a more accurate estimate, repeat this process multiple times

In [ ]:
days_ny = list(range(15180, 15185)) + list(range(15243, 15248)) + list(range(15187, 15192))
sampling_list = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

In [ ]:
rng = np.random.default_rng()
w = wte.wait_time_estimator()
w.load_index_default_fn(4092, fnbase, use_paths = True)
w.load_distances(dist_fn)
w.strategy = wte.wait_time_estimator.cruising_strategy.DRIVER_RANDOM_NODE
w.read_cruising_target_dist(0, cruising_target_dist_fn)

In [ ]:
read_trips_with_pandas = False # set to true to read trips with Pandas

In [ ]:
res = list()
for day in tqdm_notebook(days_ny):
    trips = None
    if read_trips_with_pandas:
        trips = load_trips(tripsfile_base, day, 5)
    else:
        w.read_trips_csv("{}_{}.bz2".format(tripsfile_base, day), compress = "bz2",
                 tmin = 18000, tlimit_in_day = True, length_min = 120, length_max = 7200,
                 dist_min = 500, speed_max = 100 / 3.6, clear_trips = True)
        w.read_trips_csv("{}_{}.bz2".format(tripsfile_base, day + 1), compress = "bz2",
                 tmax = 17999, tlimit_in_day = True, length_min = 120, length_max = 7200,
                 dist_min = 500, speed_max = 100 / 3.6, clear_trips = False)
        w.sort_trips()

    for pr in tqdm_notebook(sampling_list):
        (ndrivers, matched, unmatched) = process_one_day(w, rng, trips, pr)
        ntrips = matched + unmatched
        res.append({'day': day, 'pr': pr, 'ndrivers': ndrivers, 'ntrips': ntrips,
                    'pct_served': matched / ntrips})

### 5. Process the previous results
Calculate fleet size factors and plot these as a function of subsampling rate

In [ ]:
res2 = pd.DataFrame(res)
res100 = res2[res2.pr == 1.0]
res3 = res2.merge(res100, on = ['day'], suffixes= ['','100'])
res3['rdrivers'] = res3['ndrivers'] / res3['ndrivers100']
res3['factor'] = res3['rdrivers'] / res3['pr'] - 1 # fleet size factor as defined in Eq. (1)

In [ ]:
# plot all data points
res3.plot(x = 'pr', y = 'factor', kind = 'scatter')